# CONTENIDO

1.   Representacion grafica de un sistema de inventarios en Python
2.   Inventarios EOQ
3.   Inventarios News Vendor
4.   Inventarios QR
5.   Inventarios Wagner Whithin





<h1><center>Logistica</center></h1>


<h2><center>Jorge Ivan Romero</center></h2>
<h3><center>Inventarios aplicados con Python</center></h3>
<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/2048px-Python-logo-notext.svg.png"     
width="100" height="100" /></center>
<h4><center>UDES</center></h4>
<p><center>Ingenieria Industrial</center></p>
<p><center>08/04/2022</center></p>

## 1.   Representacion grafica de un sistema de inventarios en Python


In [ ]:
class almacen:
  def __init__(self, nivel_inventario):
    self.i = nivel_inventario
    self.revisa_inventario(0)
  def orden(self, Q, tiempo):
    self.revisa_inventario(tiempo)
    self.i += Q
    self.revisa_inventario(tiempo) # las ordenes son recibidas al iniciar cada periodo
  def disponible(self):
    return max(0, self.i)
  def emision(self, demanda, tiempo):
    self.revisa_inventario(tiempo)
    self.i = self.i-demanda
    self.revisa_inventario(tiempo+1) # la demanda se realiza al final de un período
  def revisa_inventario(self, tiempo):
    try:
      self.niveles.append([tiempo, self.i])
      self.en_mano.append([tiempo, self.disponible()])
    except AttributeError:
      self.niveles, self.en_mano = [[0, self.i]], [[0, self.disponible()]]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
def grafica_inventario(valores, etiqueta):
  df=pd.DataFrame({'x':np.array(valores)[:,0], 'fx':np.array(valores)[:,1]})
# grafique
  plt.xticks(range(len(valores)),range(1,len(valores)+1))
  plt.xlabel("$t$")
  plt.ylabel("items")
  plt.plot( 'x', 'fx', data=df,linestyle='-', marker='v',label=etiqueta)


Simulamos las operaciones de este sencillo sistema de inventario aprovechando el código Python en el Listado 2 y el Listado 3. El inventario inicial del almacén es de 100 unidades. La tasa de demanda del cliente es de 5 unidades por período. Simulamos N = 20 períodos. El sistema comienza con 100 unidades de inventario al comienzo del período 1; Se consumen 5 unidades de inventario en cada período; al final del período 20 (o de manera equivalente, al comienzo del período 21), el nivel de inventario del sistema es 0.

In [ ]:
inicial=100
dep=almacen(inicial)
demanda=5
N=40
for t in range(N):
  if(t==20):
    dep.orden(100,t)
  dep.emision(demanda,t)
grafica_inventario(dep.niveles,"nivel de inventario")
plt.legend()
plt.show()

In [ ]:
inv=100
dep=almacen(inv)
demanda=100/6
N=12
for t in range(N):
  if(t==6):
    dep.orden(100,t)
  dep.emision(demanda,t)
grafica_inventario(dep.niveles,"nivel de inventario")
plt.xlabel("$t$ periodos de 6 meses")
plt.axhline(60,color='r',label='reorden',linestyle='--')
plt.legend()
plt.show()

# Funciones para EOQ
## Decisiones de cantidad
$Q^{*}=\sqrt{\frac{2AD}{h}}$

$t^{*}=\frac{Q^{*}}{D}$

In [ ]:
from math import sqrt
def eoq(a:float,d:float,h:float):
  q=sqrt(2*a*d/h)
  return q
def tiempo(q:float,d:float):
  t=q/d
  return t


**Ejemplo 1**

La demanda semanal de un producto es de 1000 unidades y los artículos se retiran a una tasa constante. El costo de colocar una orden para reabastecer el inventario es de 40 dólares. El costo unitario por artículo es de 5 dólares y el costo de mantener un inventario es de $0.10 por artículo por semana.


In [ ]:
d=1000 # demanda semanal
a=40 # dolares por pedido
c=5 # dolares por articulo
h=0.10 # dolares por unidad por semana
q=eoq(a,d,h)
t=tiempo(q,d)
print('El pedido debe ser de =',q,'unidades cada',t,'semanas')
inicial=0
dep=almacen(inicial)
demanda=d*t
N=8
for t in range(N):
  dep.orden(q,t)
  dep.emision(demanda,t)
grafica_inventario(dep.niveles,"nivel de inventario")
plt.xlabel("$t$ corresponde a un periodo de 0.89 semanas")
plt.axvline(1.11,color='r',label='semana 1',linestyle='--')
plt.axvline(2.22,color='g',label='semana 2',linestyle='--')
plt.axvline(3.33,color='y',label='semana 3',linestyle='--')
plt.axvline(4.44,color='m',label='semana 4',linestyle='--')
plt.legend()
plt.show

# EOQ con faltantes

CEP con faltantes $Q^{*}=\sqrt{\frac{2AD}{h}}*\sqrt{\frac{p+h}{p}}$

Inv despues de pedido $S^{*}=\sqrt{\frac{2AD}{h}}*\sqrt{\frac{p}{p+h}}$

Maximo faltante permitido = $Q^{*}-S^{*}$

**Ejemplo 2**
La demanda semanal de un producto es de 1000 unidades y los artículos se retiran a una tasa constante. El costo de colocar una orden para reabastecer el inventario es de 40 dólares. El costo unitario por artículo es de 5 dólares y el costo de mantener un inventario es de $0.10 por artículo por semana.
Si se permiten faltantes semanales de 3 dólares por artículo, determine cuándo y cuánto debe ordenarse.

In [ ]:
d=1000.0
a=40.0
h=0.10
p=3.0
q=eoqf(a,d,h,p)
t=tiempo(q,d)
se=s(a,d,h,p)
mf=mfp(se,q)
print('Se deben ordenar',q,'unidades')
print('Se ordena cada',t,'semanas')
print('El faltante permitido es=',mf,'unidades')
print('Nos queda',se,'unidades')
inv=0
al=almacen(inv)
N=8
demanda=t*d
for t in range(N):
  al.orden(q,t)
  al.emision(demanda,t)
grafica_inventario(al.niveles,"nivel de inventario")
plt.axhline(se,color='r',label='inventario despues de entregar faltante',linestyle='--')
plt.legend()
plt.axvline(1.09,color='g',linestyle='-')
plt.axvline(2.18,color='g',linestyle='-')
plt.axvline(3.27,color='g',linestyle='-')
plt.axvline(4.36,color='g',linestyle='-')
plt.show()

In [ ]:
from math import sqrt
def eoqf(a:float,d:float,h:float,p:float):
  q=sqrt(2*a*d/h)*sqrt((p+h)/p)
  return q
def s(a:float,d:float,h:float,p:float):
  s=sqrt(2*a*d/h)*sqrt(p/(p+h))
  return s
def mfp(s:float,q:float):
  mf=q-s
  return mf

In [ ]:
inventario_inicial = 100
w = almacen(inventario_inicial)
demanda = 5 # tasa de demanda del cliente para un periodo
N = 20 # horizonte de planeacion
for t in range(N):
  w.emision(demanda, t)
grafica_inventario(w.niveles, "nivel de inventario")
plt.legend()
plt.show()

In [ ]:
inventario_inicial = 100
w = almacen(inventario_inicial)
demanda = 5 #
N = 20
for t in range(N):
  if(t == 10):
    w.orden(50, t) # colocamos una orden de 50 unidadesen el periodo 10
  w.emision(demanda, t)
grafica_inventario(w.niveles, "nivel de inventario")
plt.legend()
plt.show()

In [ ]:
inventario_inicial = 0
w = almacen(inventario_inicial)
demanda = 5 #
N = 20
for t in range(N):
  w.orden(5, t)
  w.emision(demanda, t)
grafica_inventario(w.niveles, "nivel de inventario")
plt.legend()
plt.show()

# News Vendor

In [ ]:
import scipy.integrate as integrate
from scipy.stats import norm
from scipy.optimize import minimize
class Newsvendor:
  def __init__(self, instance):
    self.mean, self.std, self.o, self.u = instance["promedio"], instance["std"],instance["co"], instance["cu"]
  def crit_frac_solution(self):
    return norm.ppf(self.u/(self.o+self.u), loc=self.mean, scale=self.std)
  def cfolf(self,Q): # función complementaria de pérdida de primer orden
    return integrate.quad(lambda x: norm.cdf(x, loc=self.mean, scale=self.std), 0,Q)[0]
  def C(self,Q): # C(Q)
    return (self.o+self.u)*self.cfolf(Q)-self.u*(Q - self.mean)
  def optC(self): # min C(Q)
    return minimize(self.C, 0, method='Nelder-Mead')

In [ ]:
mac={"co":15,"cu":50,"promedio":11.73,"std":4.74}
nv=Newsvendor(mac)
print("Q^*="+str(nv.crit_frac_solution()))
print("El costo es="+str(nv.C(nv.crit_frac_solution())))

In [ ]:
inv=0
alm=almacen(inv)
q=15.22
d=11.73
N=6
for t in range(N):
  alm.orden(q,t)
  alm.emision(d,t)
grafica_inventario(alm.niveles,'nivel de inventario')
plt.legend()
plt.show()

In [ ]:
import numpy as np
inv=0
alm=almacen(inv)
q=15.22
N=6
for t in range(N):
  alm.orden(q,t)
  de=np.random.normal(11.74, 4.74, 1)
  alm.emision(de,t)
grafica_inventario(alm.niveles,'nivel de inventario')
plt.legend()
plt.show()

# EOQ